In [1]:
import pandas as pd
import numpy as np
import xgboost
from sklearn.metrics import mean_squared_error
from sklearn.externals import joblib
import time

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

# http://xgboost.readthedocs.io/en/latest/python/python_api.html

/home/mati/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def obtener_rmse(col_true, col_pred):
    return mean_squared_error(col_true, col_pred)**0.5

In [2]:
propiedades = pd.read_csv('../../set_datos_propiedades.csv')

In [4]:
propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull()),\
                             ['place_name_encoded', 'property_type_encoded','price_aprox_usd','superficie',\
                             'Year','Month','seguridad','aire','gimnasio','cochera','pileta']]

# XGBoost

In [5]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [6]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [7]:
xgb = xgboost.XGBRegressor()
xgb.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = xgb.predict(set_pruebas.loc[:,columnas])
error = mean_squared_error(set_pruebas.price_aprox_usd, set_pruebas.resultado)
print('Error: ', error)

('Error: ', 70627765788.652054)


## Ahora que tenemos una intuicion, probamos cambiando los parametros

In [57]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [58]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

res = []

In [ ]:
max_depth = [1, 3, 7]
learning = [0.1, 0.3, 0.5]
n_estimators = [50, 100]
booster = ['gbtree', 'gblinear', 'dart']

for m in max_depth:
    for n in n_estimators:
        for l in learning:
            for b in booster:
                xgb = xgboost.XGBRegressor(n_estimators = n, max_depth = m, learning_rate = l, booster = b)
                xgb.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
                set_pruebas.loc[:,'resultado'] = xgb.predict(set_pruebas.loc[:,columnas])
                error = mean_squared_error(set_pruebas.price_aprox_usd, set_pruebas.resultado)
                res.append((m, n, l, b, error))
                print(m, n, l, b)

In [ ]:
max_depth = [7, 10]
l = 0.5
n_estimators = [300, 500]
b = 'gbtree'
for m in max_depth:
    for n in n_estimators:
        xgb = xgboost.XGBRegressor(n_estimators = n, max_depth = m, learning_rate = l, booster = b)
        xgb.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
        set_pruebas.loc[:,'resultado'] = xgb.predict(set_pruebas.loc[:,columnas])
        error = mean_squared_error(set_pruebas.price_aprox_usd, set_pruebas.resultado)
        res.append((m, n, l, b, error))
        print(m, n, l, b)

In [61]:
for r in res:
    print("Max_depth = {}, n_estimators = {}, learning_rate = {} , booster = {}, error = {}".format(r[0],r[1],r[2],r[3], r[4]))

Max_depth = 1, n_estimators = 50, learning_rate = 0.1 , booster = gbtree, error = 89451293300.6
Max_depth = 1, n_estimators = 50, learning_rate = 0.1 , booster = gblinear, error = 1.19920619905e+11
Max_depth = 1, n_estimators = 50, learning_rate = 0.1 , booster = dart, error = 89451293300.6
Max_depth = 1, n_estimators = 50, learning_rate = 0.3 , booster = gbtree, error = 87059794761.6
Max_depth = 1, n_estimators = 50, learning_rate = 0.3 , booster = gblinear, error = 1.19146619035e+11
Max_depth = 1, n_estimators = 50, learning_rate = 0.3 , booster = dart, error = 87059794761.6
Max_depth = 1, n_estimators = 50, learning_rate = 0.5 , booster = gbtree, error = 85218127548.2
Max_depth = 1, n_estimators = 50, learning_rate = 0.5 , booster = gblinear, error = 1.19071791597e+11
Max_depth = 1, n_estimators = 50, learning_rate = 0.5 , booster = dart, error = 85218127548.2
Max_depth = 1, n_estimators = 100, learning_rate = 0.1 , booster = gbtree, error = 87961505560.7
Max_depth = 1, n_estimators

In [62]:
min_error = float('inf')
tupla_min_error = ()
for r in res:
    if r[4] < min_error:
        min_error = r[4]
        tupla_min_error = r
        
print("Max_depth = {}, n_estimators = {}, learning_rate = {} , booster = {}, error = {}".\
    format(tupla_min_error[0],tupla_min_error[1],tupla_min_error[2],tupla_min_error[3], tupla_min_error[4]))

Max_depth = 10, n_estimators = 500, learning_rate = 0.5 , booster = gbtree, error = 12974389431.9


## Elegimos max_depth = 7, n_estimators = 100, learning_rate = 0.5 y booster = gbtree

In [15]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [16]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [17]:
m = 7
l = 0.5
n = 100
b = 'gbtree'

xgb = xgboost.XGBRegressor(n_estimators = n, max_depth = m, learning_rate = l, booster = b)
xgb.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = xgb.predict(set_pruebas.loc[:,columnas])
error = mean_squared_error(set_pruebas.price_aprox_usd, set_pruebas.resultado)
print(error)

34462614151.3


## Elegimos max_depth = 10, n_estimators = 500, learning_rate = 0.5 y booster = gbtree

In [47]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [48]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [49]:
m = 10
l = 0.5
n = 500
b = 'gbtree'

xgb = xgboost.XGBRegressor(n_estimators = n, max_depth = m, learning_rate = l, booster = b)
xgb.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = xgb.predict(set_pruebas.loc[:,columnas])
error = mean_squared_error(set_pruebas.price_aprox_usd, set_pruebas.resultado)
print(error)

12974389431.9


# calculo con los verdaderos datos a analizar

In [50]:
analizar = pd.read_csv("../properati_dataset_modificado.csv")

In [51]:
analizar.loc[:,'price_usd'] = xgb.predict(analizar.loc[:,columnas])

In [52]:
analizar.price_usd.describe()

count    1.416600e+04
mean     2.380542e+05
std      4.528047e+05
min     -8.993336e+05
25%      1.032099e+05
50%      1.543094e+05
75%      2.629287e+05
max      4.404158e+07
Name: price_usd, dtype: float64

In [53]:
# Las predicciones con precio negativo las convierto en valores positivos
analizar.loc[:,'price_usd'] = analizar.loc[:,"price_usd"].apply(lambda x: abs(x))

In [54]:
analizar.price_usd.describe()

count    1.416600e+04
mean     2.393106e+05
std      4.521419e+05
min      2.949538e+02
25%      1.037648e+05
50%      1.547995e+05
75%      2.633302e+05
max      4.404158e+07
Name: price_usd, dtype: float64

In [55]:
resultado = analizar.loc[:,['id','price_usd']]

In [56]:
resultado.to_csv('resultados/XGBoost_resultados_2.csv', index = False)

# Ahora uso lat y lon en lugar de place name

In [3]:
propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()),\
                             ['lat', 'lon', 'property_type_encoded','price_aprox_usd','superficie',\
                             'Year','Month','seguridad','aire','gimnasio','cochera','pileta']]

In [4]:
columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [5]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [6]:
xgb = xgboost.XGBRegressor()
xgb.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = xgb.predict(set_pruebas.loc[:,columnas])
error = mean_squared_error(set_pruebas.price_aprox_usd, set_pruebas.resultado)
print('Error: ', error)

('Error: ', 52794471961.235115)


In [7]:
res = []

In [ ]:
max_depth = [1, 3, 7]
learning = [0.1, 0.3, 0.5]
n_estimators = [50, 100]
booster = ['gbtree', 'gblinear', 'dart']

for m in max_depth:
    for n in n_estimators:
        for l in learning:
            for b in booster:
                xgb = xgboost.XGBRegressor(n_estimators = n, max_depth = m, learning_rate = l, booster = b)
                xgb.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
                set_pruebas.loc[:,'resultado'] = xgb.predict(set_pruebas.loc[:,columnas])
                error = mean_squared_error(set_pruebas.price_aprox_usd, set_pruebas.resultado)
                res.append((m, n, l, b, error))
                print(m, n, l, b)

In [10]:
min_error = float('inf')
tupla_min_error = ()
for r in res:
    if r[4] < min_error:
        min_error = r[4]
        tupla_min_error = r
        
print("Max_depth = {}, n_estimators = {}, learning_rate = {} , booster = {}, error = {}".\
    format(tupla_min_error[0],tupla_min_error[1],tupla_min_error[2],tupla_min_error[3], tupla_min_error[4]))

Max_depth = 7, n_estimators = 100, learning_rate = 0.5 , booster = gbtree, error = 18680682185.9


In [ ]:
max_depth = [7, 10]
l = 0.5
n_estimators = [300, 500]
b = 'gbtree'
for m in max_depth:
    for n in n_estimators:
        xgb = xgboost.XGBRegressor(n_estimators = n, max_depth = m, learning_rate = l, booster = b)
        xgb.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
        set_pruebas.loc[:,'resultado'] = xgb.predict(set_pruebas.loc[:,columnas])
        error = mean_squared_error(set_pruebas.price_aprox_usd, set_pruebas.resultado)
        res.append((m, n, l, b, error))
        print(m, n, l, b)

In [12]:
for r in res:
    print("Max_depth = {}, n_estimators = {}, learning_rate = {} , booster = {}, error = {}".format(r[0],r[1],r[2],r[3], r[4]))

Max_depth = 1, n_estimators = 50, learning_rate = 0.1 , booster = gbtree, error = 77881015910.2
Max_depth = 1, n_estimators = 50, learning_rate = 0.1 , booster = gblinear, error = 1.04636623742e+11
Max_depth = 1, n_estimators = 50, learning_rate = 0.1 , booster = dart, error = 77881015910.2
Max_depth = 1, n_estimators = 50, learning_rate = 0.3 , booster = gbtree, error = 71678787504.9
Max_depth = 1, n_estimators = 50, learning_rate = 0.3 , booster = gblinear, error = 1.04043928525e+11
Max_depth = 1, n_estimators = 50, learning_rate = 0.3 , booster = dart, error = 71678787504.9
Max_depth = 1, n_estimators = 50, learning_rate = 0.5 , booster = gbtree, error = 70326846666.8
Max_depth = 1, n_estimators = 50, learning_rate = 0.5 , booster = gblinear, error = 1.04006153461e+11
Max_depth = 1, n_estimators = 50, learning_rate = 0.5 , booster = dart, error = 70326846666.8
Max_depth = 1, n_estimators = 100, learning_rate = 0.1 , booster = gbtree, error = 73611591194.6
Max_depth = 1, n_estimators

In [13]:
min_error = float('inf')
tupla_min_error = ()
for r in res:
    if r[4] < min_error:
        min_error = r[4]
        tupla_min_error = r
        
print("Max_depth = {}, n_estimators = {}, learning_rate = {} , booster = {}, error = {}".\
    format(tupla_min_error[0],tupla_min_error[1],tupla_min_error[2],tupla_min_error[3], tupla_min_error[4]))

Max_depth = 10, n_estimators = 500, learning_rate = 0.5 , booster = gbtree, error = 2249870876.26


In [32]:
# Obtengo el error con el set de entrenamiento
mean_squared_error(xgb.predict(set_entrenamiento_datos), set_entrenamiento_resultado)

951447283.6083287

# calculo con los verdaderos datos a analizar

In [16]:
analizar = pd.read_csv("../properati_dataset_modificado.csv")

In [17]:
analizar.loc[:,'price_usd'] = xgb.predict(analizar.loc[:,columnas])

In [18]:
analizar.price_usd.describe()

count    1.416600e+04
mean     2.466999e+05
std      6.413061e+05
min     -4.142394e+05
25%      1.012079e+05
50%      1.538635e+05
75%      2.615367e+05
max      4.942000e+07
Name: price_usd, dtype: float64

In [19]:
# Las predicciones con precio negativo las convierto en valores positivos
analizar.loc[:,'price_usd'] = analizar.loc[:,"price_usd"].apply(lambda x: abs(x))

In [20]:
analizar.price_usd.describe()

count    1.416600e+04
mean     2.476073e+05
std      6.409562e+05
min      1.697878e+02
25%      1.015205e+05
50%      1.540672e+05
75%      2.617388e+05
max      4.942000e+07
Name: price_usd, dtype: float64

In [21]:
resultado = analizar.loc[:,['id','price_usd']]

In [22]:
resultado.to_csv('resultados/XGBoost_resultados_latlon.csv', index = False)

# Usando muchos estimadores

In [2]:
sup_min = 10
sup_max = 300
lat_min = -35
lat_max = -34
lon_min = -59
lon_max = -58
precio_min = 30000
precio_max = 3000000

In [3]:
propiedades = pd.read_csv('../../datos_con_fecha.csv')

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2017)\
                        & (propiedades.price_aprox_usd <= precio_max) & (propiedades.price_aprox_usd >= precio_min) &\
                         (propiedades.superficie <= sup_max) & (propiedades.superficie >= sup_min) &\
                      (propiedades.lat <= lat_max) & (propiedades.lat >= lat_min) &\
                       (propiedades.lon <= lon_max) & (propiedades.lon >= lon_min),:]

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', \
            'pileta', 'cochera','transporte','comercio', 'servicios', 'doble piso', 'jardin','fecha']

set_entrenamiento_datos = propiedades.loc[:,columnas]
set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']

In [5]:
inicio = time.strftime("%X")

xgb = xgboost.XGBRegressor(n_estimators = 5000, max_depth = 50, learning_rate = 0.5)

xgb.fit(set_entrenamiento_datos, set_entrenamiento_resultado)

fin = time.strftime("%X")

print("Tiempo: {} --- {} ".format(inicio,fin))

Tiempo: 22:32:24 --- 01:42:32 


In [12]:
analizar = pd.read_csv("../properati_dataset_modificado2.csv")

analizar.loc[:,'price_usd'] = xgb.predict(analizar.loc[:,columnas])
analizar.loc[:,'price_usd'] = analizar.loc[:,"price_usd"].apply(lambda x: abs(x))
resultado = analizar.loc[:,['id','price_usd']]

resultado.to_csv('resultados/XGBoost_5kEst_confecha.csv', index = False)
joblib.dump(xgb, 'algoritmos/XGBoost_5kEst_confecha.pkl')

['algoritmos/XGBoost_5kEst_confecha.pkl']

## Con 15k

In [ ]:
sup_min = 10
sup_max = 300
lat_min = -35
lat_max = -34
lon_min = -59
lon_max = -58
precio_min = 30000
precio_max = 3000000

In [ ]:
propiedades = pd.read_csv('../../datos_con_fecha.csv')

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2017)\
                        & (propiedades.price_aprox_usd <= precio_max) & (propiedades.price_aprox_usd >= precio_min) &\
                         (propiedades.superficie <= sup_max) & (propiedades.superficie >= sup_min) &\
                      (propiedades.lat <= lat_max) & (propiedades.lat >= lat_min) &\
                       (propiedades.lon <= lon_max) & (propiedades.lon >= lon_min),:]

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', \
            'pileta', 'cochera','transporte','comercio', 'servicios', 'doble piso', 'jardin','fecha']

set_entrenamiento_datos = propiedades.loc[:,columnas]
set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']

In [ ]:
inicio = time.strftime("%X")

xgb = xgboost.XGBRegressor(n_estimators = 15000, max_depth = 50, learning_rate = 0.5)

xgb.fit(set_entrenamiento_datos, set_entrenamiento_resultado)

fin = time.strftime("%X")

print("Tiempo: {} --- {} ".format(inicio,fin))

In [ ]:
analizar = pd.read_csv("../properati_dataset_modificado2.csv")

analizar.loc[:,'price_usd'] = xgb.predict(analizar.loc[:,columnas])
analizar.loc[:,'price_usd'] = analizar.loc[:,"price_usd"].apply(lambda x: abs(x))
resultado = analizar.loc[:,['id','price_usd']]

resultado.to_csv('resultados/XGBoost_15kEst_confecha.csv', index = False)
joblib.dump(xgb, 'algoritmos/XGBoost_15kEst_confecha.pkl')

# Usando muchos estimadores sin fecha

In [ ]:
sup_min = 10
sup_max = 300
lat_min = -35
lat_max = -34
lon_min = -59
lon_max = -58
precio_min = 30000
precio_max = 3000000

In [ ]:
propiedades = pd.read_csv('../../datos_con_fecha.csv')

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2017)\
                        & (propiedades.price_aprox_usd <= precio_max) & (propiedades.price_aprox_usd >= precio_min) &\
                         (propiedades.superficie <= sup_max) & (propiedades.superficie >= sup_min) &\
                      (propiedades.lat <= lat_max) & (propiedades.lat >= lat_min) &\
                       (propiedades.lon <= lon_max) & (propiedades.lon >= lon_min),:]

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', \
            'pileta', 'cochera','transporte','comercio', 'servicios', 'doble piso', 'jardin']

set_entrenamiento_datos = propiedades.loc[:,columnas]
set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']

In [ ]:
inicio = time.strftime("%X")

xgb = xgboost.XGBRegressor(n_estimators = 15000, max_depth = 50, learning_rate = 0.5)

xgb.fit(set_entrenamiento_datos, set_entrenamiento_resultado)

fin = time.strftime("%X")

print("Tiempo: {} --- {} ".format(inicio,fin))

In [ ]:
analizar = pd.read_csv("../properati_dataset_modificado2.csv")

analizar.loc[:,'price_usd'] = xgb.predict(analizar.loc[:,columnas])
analizar.loc[:,'price_usd'] = analizar.loc[:,"price_usd"].apply(lambda x: abs(x))
resultado = analizar.loc[:,['id','price_usd']]

resultado.to_csv('resultados/XGBoost_15kEst_sinfecha.csv', index = False)
joblib.dump(xgb, 'algoritmos/XGBoost_15kEst_sinfecha.pkl')